<a href="https://colab.research.google.com/github/eduardotvn/Processador-de-Reclamacoes/blob/main/Previsor_de_Contexto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [343]:
import pandas as pd

In [344]:
df = pd.read_csv('/content/Banco de Reclamações - Página1.csv', sep = ',')
df = df.drop_duplicates()
df = df.dropna()


In [326]:
!python -m nltk.downloader stopwords

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [345]:
import re
from nltk.corpus import stopwords

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    stop_words = set(stopwords.words('portuguese'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

df['Frase'] = df['Frase'].apply(preprocess_text)


In [346]:
df

,Frase,Categoria,Label
0,consigo fazer tudo preciso,Tempo,0
1,dia horas suficientes,Tempo,0
2,sempre correndo contra relógio,Tempo,0
3,sobra tempo mim,Tempo,0
4,sobrecarregado tantas tarefas,Tempo,0
...,...,...,...
853,nada impede,Nada,5
854,medo nada,Nada,5
855,nada irá impedir,Nada,5
856,momento nada,Nada,5


In [347]:
from sklearn.model_selection import train_test_split

X = df['Frase']
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [348]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [349]:
X_test_vec

<140x610 sparse matrix of type '<class 'numpy.float64'>'
	with 339 stored elements in Compressed Sparse Row format>

In [350]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(random_state=42)
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy:.2f}")


Acurácia: 0.74


In [357]:
nova_frase = [" nada a não ser dinheiro"]

nova_frase_vec = vectorizer.transform(nova_frase)

tópico_predito = model.predict(nova_frase_vec)

In [358]:
contexto = {0 : 'Tempo', 1 : 'Dinheiro', 2 : 'Conhecimento', 3: 'Insegurança', 4: 'Outros', 5:'Nada'}

In [359]:
print(f"Tópico sugerido para a frase '{nova_frase[0]}': {contexto[tópico_predito[0]]}")


Tópico sugerido para a frase ' nada a não ser dinheiro': Dinheiro


In [360]:
dataset_teste = pd.read_csv('/content/Pesquisa com Leads.csv', sep = ',')

In [361]:
lista_teste = dataset_teste['O que te impede de realizar esse sonho?'].fillna('').tolist()

In [362]:
predicoes_totais = []

for frase in lista_teste:
  input = [frase]
  tópico_predito = model.predict(vectorizer.transform(input))
  predicoes_totais.append(contexto[tópico_predito[0]])

In [363]:
dataset_teste['Topico_Reclamacoes'] = predicoes_totais

In [370]:
dataset_teste[dataset_teste['Topico_Reclamacoes'] == 'Insegurança'][['O que te impede de realizar esse sonho?', 'Topico_Reclamacoes']][:50]

,O que te impede de realizar esse sonho?,Topico_Reclamacoes
4,Eu mesma,Insegurança
10,Medos,Insegurança
31,Medo,Insegurança
54,meu medo rsrs,Insegurança
90,"O medo de pisar, de mudar",Insegurança
98,Medo,Insegurança
105,O medo,Insegurança
110,Medo de sair do CLT,Insegurança
151,Medo do futuro incerto.,Insegurança
177,Medo de ser enganado,Insegurança


In [371]:
dataset_teste[['O que te impede de realizar esse sonho?', 'Topico_Reclamacoes']][:50]

,O que te impede de realizar esse sonho?,Topico_Reclamacoes
0,Ainda não encontrei uma forma de escalar e aut...,Conhecimento
1,Encontrar alguém que realmente entregue o que ...,Outros
2,situação financeira,Dinheiro
3,Tempo,Tempo
4,Eu mesma,Insegurança
5,Falta de dinheiro,Dinheiro
6,não tenho impeditivo,Outros
7,Nao sei te responder,Conhecimento
8,teste,Outros
9,Não saber como fazer,Conhecimento
